# <center style="font-family: consolas; font-size: 32px; font-weight: bold;"> 🌋 Vesuvius Challenge - 📜 Ink Detection - Exploratory Data Analysis</center>
<p><center style="color:#949494; font-family: consolas; font-size: 20px;">Resurrect an ancient library from the ashes of a volcano</center></p>

***

# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">(ಠಿ⁠_⁠ಠ) Overview</center>

<p style="font-family: consolas; font-size: 16px;">⚪ The goal of this competition is to <b>detect ink from 3D X-ray scans of ancient scrolls</b> from a library buried by the eruption of Vesuvius nearly 2000 years ago. The scrolls were carbonized due to the heat of the volcano and are now <b>impossible to open without breaking them</b>.</p>

<p style="font-family: consolas; font-size: 16px;">⚪ The competition offers a <b>grand prize of 150,000 USD to the first team that can read these scrolls from a 3D X-ray scan</b>. The competition hosts the Ink Detection progress prize, which is about the sub-problem of detecting ink from 3D X-ray scans of fragments of papyrus that became detached from some of the excavated scrolls.</p>

<p style="font-family: consolas; font-size: 16px;">⚪ The ink used in the Herculaneum scrolls does not show up readily in X-ray scans, but machine learning models can detect it. The dataset contains 3D X-ray scans of four fragments at 4µm resolution, made using a particle accelerator, as well as infrared photographs of the surface of the fragments showing visible ink.</p>

<p style="font-family: consolas; font-size: 16px;">⚪ Hand-labeled binary masks indicating the presence of ink in the photographs are also provided.</p>

#### <a id="top"></a>
# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">

* [0. Import all dependencies](#0)
* [1. Overview directories](#1)
* [2. Overview the 1st volume of the train/ directory](#2)
    * [2.1 Consider mask.png image](#2.1)
    * [2.2 Consider ir.png image](#2.2)
    * [2.3 Consider inklabels.png image](#2.3)
    * [2.4 Consider inklabels_rle.csv file](#2.4)
    * [2.5 Consider surface_volumes/ directory](#2.5)
* [3. Overview the 2nd volume of the train/ directory](#3)
    * [3.1 Consider mask.png image](#3.1)
    * [3.2 Consider ir.png image](#3.2)
    * [3.3 Consider inklabels.png image](#3.3)
    * [3.4 Consider inklabels_rle.csv file](#3.4)
    * [3.5 Consider surface_volumes/ directory](#3.5)
* [4. Overview the 3rd volume of the train/ directory](#4)
    * [4.1 Consider mask.png image](#4.1)
    * [4.2 Consider ir.png image](#4.2)
    * [3.3 Consider inklabels.png image](#4.3)
    * [4.4 Consider inklabels_rle.csv file](#4.4)
    * [4.5 Consider surface_volumes/ directory](#4.5)
* [5. Overview volumes of the test/ directory](#5)
    * [5.1 Consider mask.png images](#5.1)

<a id="0"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 0. Install & Import all dependencies </b></div>

In [ ]:
!pip install celluloid -q

In [ ]:
import os
import gc
import matplotlib.pyplot as plt
from typing import Union, Optional
from PIL import Image
from IPython.display import HTML, display

import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
from celluloid import Camera

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
plt.rcParams['figure.dpi'] = 350
plt.style.use('dark_background')

In [ ]:
class CFG:
    layers_count: int = 65
    train_volumes_path_template: str = "/kaggle/input/vesuvius-challenge-ink-detection/train/{}/surface_volume"
    test_volumes_path_template: str = "/kaggle/input/vesuvius-challenge-ink-detection/test/{}/surface_volume"
    tif_template: str = "{:02d}.tif"

In [ ]:
def load_volume(volume_path: str, disable_tqdm: bool = False) -> np.ndarray:
    volume = []
    for i in tqdm(range(CFG.layers_count), disable=disable_tqdm):
        img = Image.open(f"{volume_path}/{CFG.tif_template.format(i)}")
        arr = np.array(img)
        volume.append(arr)
    
    return volume

In [ ]:
def animate_volume(
    volume: Union[np.ndarray, str], 
    ir_photo: Optional[np.ndarray] = None,
    inklabels: Optional[np.ndarray] = None,
) -> None:
    
    plt.rcParams['figure.dpi'] = 350
    plt.style.use('dark_background')
    
    if isinstance(volume, str):
        volume = load_volume(volume, disable_tqdm=True)

    # creating figure subplot
    if ir_photo and inklabels:
        fig, (ax_ir, ax, ax_il) = plt.subplots(1, 3)
    elif ir_photo:
        fig, (ax, ax_ir) = plt.subplots(1, 2)
    elif inklabels:
        fig, (ax, ax_il) = plt.subplots(1, 2)
    else:
        fig, ax = plt.subplots()
    
    camera = Camera(fig) # define the camera that gets the fig we'll plot
    for i in range(CFG.layers_count):
        ax.axis('off')
        ax.text(
            0.5, 1.08, f"Layer {i+1}/{CFG.layers_count}", fontweight='bold', fontsize=18,
            transform=ax.transAxes, horizontalalignment='center'
        )
        ax.imshow(volume[0], cmap='gray') # plot volume layer
        
        if ir_photo:
            ax_ir.imshow(ir_photo, cmap='gray')
            ax_ir.axis('off')
            ax_ir.text(
                0.5, -0.13, "Infrared photo", style='italic', fontsize=15,
                transform=ax_ir.transAxes, horizontalalignment='center'
            )
        
        if inklabels:
            ax_il.imshow(inklabels, cmap='gray')
            ax_il.axis('off')
            ax_il.text(
                0.5, -0.13, "Ink labels", style='italic', fontsize=15,
                transform=ax_il.transAxes, horizontalalignment='center'
            )
        
        
        camera.snap() # the camera takes a snapshot of the plot
        
        del volume[0]
        gc.collect()

    plt.close(fig) # close figure

    animation = camera.animate() # get plt animation
    
    fix_video_adjust = '<style> video {margin: 0px; padding: 0px; width:100%; height:auto;} </style>'
    display(
        HTML(fix_video_adjust + animation.to_html5_video())
    ) # displaying the animation
    
    del camera
    del animation
    gc.collect()

In [ ]:
def get_concat_v(im1: Image, im2: Image) -> Image:
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Overview directories</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ <code>[train/test]/[fragment_id]/surface_volume/[image_id].tif</code> slices from the 3d x-ray surface volume. Each file contains a greyscale slice in the z-direction. Each fragment contains <b>65 slices</b>. Combined this image stack gives us <code>width * height * 65</code> number of voxels per fragment. You can expect two fragments in the hidden test set, which together are roughly the same as the first training fragment. The sample slices available to download in the test folders are simply copied from training fragment one.</p>

<p style="font-family: consolas; font-size: 16px;">⚪ <code>train/</code> directory has <b>3</b> 3d x-ray surface volumes.</p>

In [ ]:
sorted(os.listdir("/kaggle/input/vesuvius-challenge-ink-detection/train"))

<p style="font-family: consolas; font-size: 16px;">⚪ <code>test/</code> directory has <b>3</b> 3d x-ray surface volumes. Which together are roughly the same as the <b>1st training fragment</b></p>

In [ ]:
sorted(os.listdir("/kaggle/input/vesuvius-challenge-ink-detection/test"))

<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Overview the 1st volume of the <i>train/</i> directory</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Each of train volumes has next files: </p>

* <p style="font-family: consolas; font-size: 16px;"><code>mask.png</code> — a binary mask of which pixels contain data.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>ir.png</code> — the infrared photo on which the binary mask is based.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>inklabels.png</code> — a binary mask of the ink vs no-ink labels.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>inklabels_rle.csv</code> — a run-length-encoded version of the labels, generated using this script. This is the same format as you should make your submission in.</p>

In [ ]:
os.listdir("/kaggle/input/vesuvius-challenge-ink-detection/train/1")

<a id="2.1"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 2.1 Consider <i>mask.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Let's load the mask</p>

In [ ]:
train_1_mask = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/1/mask.png")

In [ ]:
print(f"Mask image size: {color.BOLD}{color.CYAN}{train_1_mask.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ And plot it</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_1_mask);

<p style="font-family: consolas; font-size: 16px;">⚪ OK there is nothing interesting in the mask, so let's take a look at <code>ir.png</code> image.</p>

<a id="2.2"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 2.2 Consider <i>ir.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Load the infrared image</p>

In [ ]:
train_1_ir = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/1/ir.png")

In [ ]:
print(f"Infrared image size: {color.BOLD}{color.CYAN}{train_1_ir.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ As you can see in the image there are some fragments of words. The approximate number of letters on the fragment is about 20.</p>

<p style="font-family: consolas; font-size: 16px;">🔴 It is important to note that there is no infrared image in the test dataset, only a mask and 65 volume layers from a 3d x-ray.</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_1_ir, cmap='gray');

<a id="2.3"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 2.3 Consider <i>inklabels.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Load <code>inklabels.png</code> image.</p>

In [ ]:
train_1_inklabels = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/1/inklabels.png")

In [ ]:
print(f"Inklabels image size: {color.BOLD}{color.CYAN}{train_1_inklabels.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ You are supposed to use it as a prediction reference. Thus, you predict a binary semantic mask from 3D X-ray 65 volume layers and check its intersection with the inklabels mask.</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_1_inklabels, cmap='gray');

<p style="font-family: consolas; font-size: 16px;">⚪ Let's see the distribution of ink on the first training instance. To calculate the distribution, I consider it optimal to first take a mask, and only after that calculate the distribution, since it is in this way that it is assumed that training and inference will take place.</p>

In [ ]:
# conver to np.ndarray
train_1_inklabels_np = np.array(train_1_inklabels)
train_1_mask_np = np.array(train_1_mask, dtype=np.bool8)

# reassign new value for empty space 
train_1_inklabels_np[np.logical_not(train_1_mask_np)] = 2

# get counts for each label
train_1_pix_counts = np.unique(
    train_1_inklabels_np, 
    return_counts=True
)[1]

<p style="font-family: consolas; font-size: 16px;">⚪ On visualization you will see next 3 labels: </p>

* <p style="font-family: consolas; font-size: 16px;"><code>No ink</code> — is inside the mask, but is not ink.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>Ink</code> — is inside the mask and is ink.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>Empty space</code> — is outside the mask.</p>

In [ ]:
fig = px.bar(
    x=["No ink", "Ink", "Empty space"], y=train_1_pix_counts,
    color_discrete_sequence=['darkgoldenrod']
)

fig.update_layout(
    xaxis_title="Label", yaxis_title="Pixel count",
    title={
        'text': "Distribution of Ink on the 1st train instance",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)
fig.show()

<a id="2.4"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 2.4 Consider <i>inklabels_rle.csv</i> file</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ There is nothing interesting in this file. It's just a run-length-encoded version of the <code>inklabels.png</code> image, generated using <a href="https://gist.github.com/janpaul123/ca3477c1db6de4346affca37e0e3d5b0"><strong>this script</strong></a>.</p>

<p style="font-family: consolas; font-size: 16px;">⚪ It's important to note that this is in the same format as you should make your <a href="https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/overview/evaluation"><strong>submission in</strong></a>.</p>

In [ ]:
pd.read_csv("/kaggle/input/vesuvius-challenge-ink-detection/train/1/inklabels_rle.csv")

<a id="2.5"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 2.5 Consider <i>surface_volumes/</i> directory</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ As mentioned above these are slices from the 3d x-ray surface volume. Each file contains a <b>greyscale slice in the z-direction</b>. Each fragment contains <b>65 slices</b>. Combined this image stack gives us <code>65 * height * width</code> number of voxels per fragment.</p>

In [ ]:
first_train_volume = CFG.train_volumes_path_template.format(1)

In [ ]:
train_volume_1 = load_volume(first_train_volume)

In [ ]:
print(f"1st train volume size: {color.BOLD}{color.CYAN}{train_volume_1[0].shape}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ The whole volume is huge, let's see exactly how.</p>

In [ ]:
bytes_size = CFG.layers_count * train_volume_1[0].size * train_volume_1[0].itemsize
print(f"Memory size of 1st train volume in bytes: {color.BOLD}{color.PURPLE}{bytes_size}{color.END} B")
print(f"Memory size of 1st train volume in gigabytes: {color.BOLD}{color.PURPLE}{bytes_size / 1024**3}{color.END} GB")

<p style="font-family: consolas; font-size: 16px;">⚪ For a better understanding of how the layers relate to each other, I made an <b>animation with the transition from one layer to another</b>. And so for all 65 layers.</p>

<p style="font-family: consolas; font-size: 16px;">⚪ As noted in the description of this competition, it is visually almost impossible to notice any ink on these images. But together they form a coherent picture, which, I hope, can be picked up very easily by a neural network.</p>

In [ ]:
animate_volume(train_volume_1)

<p style="font-family: consolas; font-size: 16px;">⚪ Let's animate volume right next to its infrared image. So you can look at some features on the fly!</p>

In [ ]:
animate_volume(first_train_volume, ir_photo=train_1_ir)

<p style="font-family: consolas; font-size: 16px;">⚪ And let's animate volume next to its ink labels mask.</p>

In [ ]:
# animate_volume(first_train_volume, inklabels=train_1_inklabels)

In [ ]:
# animate_volume(first_train_volume, train_1_ir, train_1_inklabels)

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Overview the 2nd volume of the <i>train/</i> directory</b></div>

<a id="3.1"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.1 Consider <i>mask.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Let's load the mask</p>

In [ ]:
train_2_mask = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/2/mask.png")

In [ ]:
print(f"Mask image size: {color.BOLD}{color.CYAN}{train_2_mask.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ And plot it</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_2_mask);

<p style="font-family: consolas; font-size: 16px;">⚪ OK there is nothing interesting in the mask, so let's take a look at <code>ir.png</code> image.</p>

<a id="3.2"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.2 Consider <i>ir.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Load the infrared image</p>

In [ ]:
train_2_ir = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/2/ir.png")

In [ ]:
print(f"Infrared image size: {color.BOLD}{color.CYAN}{train_2_ir.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ As you can see in the image there are some fragments of words. The approximate number of letters on the fragment is about 65.</p>

<p style="font-family: consolas; font-size: 16px;">🔴 It is important to note that there is no infrared image in the test dataset, only a mask and 65 volume layers from a 3d x-ray.</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_2_ir, cmap='gray');

<a id="3.3"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.3 Consider <i>inklabels.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Load <code>inklabels.png</code> image.</p>

In [ ]:
train_2_inklabels = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/2/inklabels.png")

In [ ]:
print(f"Inklabels image size: {color.BOLD}{color.CYAN}{train_2_inklabels.size}{color.END}")

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_2_inklabels, cmap='gray');

<p style="font-family: consolas; font-size: 16px;">⚪ Let's see the distribution of ink on the second training instance. To calculate the distribution, I consider it optimal to first take a mask, and only after that calculate the distribution, since it is in this way that it is assumed that training and inference will take place.</p>

In [ ]:
# conver to np.ndarray
train_2_inklabels_np = np.array(train_2_inklabels)
train_2_mask_np = np.array(train_2_mask, dtype=np.bool8)

# reassign new value for empty space 
train_2_inklabels_np[np.logical_not(train_2_mask_np)] = 2

# get counts for each label
train_2_pix_counts = np.unique(
    train_2_inklabels_np, 
    return_counts=True
)[1]

<p style="font-family: consolas; font-size: 16px;">⚪ On visualization you will see next 3 labels: </p>

* <p style="font-family: consolas; font-size: 16px;"><code>No ink</code> — is inside the mask, but is not ink.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>Ink</code> — is inside the mask and is ink.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>Empty space</code> — is outside the mask.</p>

In [ ]:
fig = px.bar(
    x=["No ink", "Ink", "Empty space"], y=train_2_pix_counts,
    color_discrete_sequence=['cadetblue']
)

fig.update_layout(
    xaxis_title="Label", yaxis_title="Pixel count",
    title={
        'text': "Distribution of Ink on the 2nd train instance",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)
fig.show()

<a id="3.4"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.4 Consider <i>inklabels_rle.csv</i> file</b></div>

In [ ]:
pd.read_csv("/kaggle/input/vesuvius-challenge-ink-detection/train/2/inklabels_rle.csv")

<a id="3.5"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.5 Consider <i>surface_volumes/</i> directory</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ As mentioned above these are slices from the 3d x-ray surface volume. Each file contains a <b>greyscale slice in the z-direction</b>. Each fragment contains <b>65 slices</b>. Combined this image stack gives us <code>65 * height * width</code> number of voxels per fragment.</p>

In [ ]:
second_train_volume = CFG.train_volumes_path_template.format(2)

In [ ]:
train_volume_2 = load_volume(second_train_volume)

In [ ]:
print(f"2st train volume size: {color.BOLD}{color.CYAN}{train_volume_2[0].shape}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ The whole volume is huge, let's see exactly how.</p>

In [ ]:
bytes_size = CFG.layers_count * train_volume_2[0].size * train_volume_2[0].itemsize
print(f"Memory size of 2st train volume in bytes: {color.BOLD}{color.PURPLE}{bytes_size}{color.END} B")
print(f"Memory size of 2st train volume in gigabytes: {color.BOLD}{color.PURPLE}{bytes_size / 1024**3}{color.END} GB")

<p style="font-family: consolas; font-size: 16px;">⚪ For a better understanding of how the layers relate to each other, I made an <b>animation with the transition from one layer to another</b>. And so for all 65 layers.</p>

In [ ]:
animate_volume(train_volume_2)

<p style="font-family: consolas; font-size: 16px;">⚪ Let's animate volume right next to its infrared image. So you can look at some features on the fly!</p>

In [ ]:
animate_volume(second_train_volume, ir_photo=train_2_ir)

<p style="font-family: consolas; font-size: 16px;">⚪ And let's animate volume next to its ink labels mask.</p>

In [ ]:
# animate_volume(second_train_volume, inklabels=train_2_inklabels)

<a id="4"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 4. Overview the 3rd volume of the <i>train/</i> directory</b></div>

<a id="4.1"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 4.1 Consider <i>mask.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Let's load the mask</p>

In [ ]:
train_3_mask = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/3/mask.png")

In [ ]:
print(f"Mask image size: {color.BOLD}{color.CYAN}{train_3_mask.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ And plot it</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_3_mask);

<p style="font-family: consolas; font-size: 16px;">⚪ OK there is nothing interesting in the mask, so let's take a look at <code>ir.png</code> image.</p>

<a id="4.2"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 4.2 Consider <i>ir.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Load the infrared image</p>

In [ ]:
train_3_ir = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/3/ir.png")

In [ ]:
print(f"Infrared image size: {color.BOLD}{color.CYAN}{train_3_ir.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ As you can see in the image there are some fragments of words. The approximate number of letters on the fragment is about 15.</p>

<p style="font-family: consolas; font-size: 16px;">🔴 It is important to note that there is no infrared image in the test dataset, only a mask and 65 volume layers from a 3d x-ray.</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_3_ir, cmap='gray');

<a id="4.3"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 4.3 Consider <i>inklabels.png</i> image</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Load <code>inklabels.png</code> image.</p>

In [ ]:
train_3_inklabels = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/3/inklabels.png")

In [ ]:
print(f"Inklabels image size: {color.BOLD}{color.CYAN}{train_3_inklabels.size}{color.END}")

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(train_3_inklabels, cmap='gray');

<p style="font-family: consolas; font-size: 16px;">⚪ Let's see the distribution of ink on the third training instance. To calculate the distribution, I consider it optimal to first take a mask, and only after that calculate the distribution, since it is in this way that it is assumed that training and inference will take place.</p>

In [ ]:
# conver to np.ndarray
train_3_inklabels_np = np.array(train_3_inklabels)
train_3_mask_np = np.array(train_3_mask, dtype=np.bool8)

# reassign new value for empty space 
train_3_inklabels_np[np.logical_not(train_3_mask_np)] = 2

# get counts for each label
train_3_pix_counts = np.unique(
    train_3_inklabels_np, 
    return_counts=True
)[1]

<p style="font-family: consolas; font-size: 16px;">⚪ On visualization you will see next 3 labels: </p>

* <p style="font-family: consolas; font-size: 16px;"><code>No ink</code> — is inside the mask, but is not ink.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>Ink</code> — is inside the mask and is ink.</p>
* <p style="font-family: consolas; font-size: 16px;"><code>Empty space</code> — is outside the mask.</p>

In [ ]:
fig = px.bar(
    x=["No ink", "Ink", "Empty space"], y=train_3_pix_counts,
    color_discrete_sequence=['darkslateblue']
)

fig.update_layout(
    xaxis_title="Label", yaxis_title="Pixel count",
    title={
        'text': "Distribution of Ink on the 3rd train instance",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)
fig.show()

<a id="4.4"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 4.4 Consider <i>inklabels_rle.csv</i> file</b></div>

In [ ]:
pd.read_csv("/kaggle/input/vesuvius-challenge-ink-detection/train/3/inklabels_rle.csv")

<a id="4.5"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 4.5 Consider <i>surface_volumes/</i> directory</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ As mentioned above these are slices from the 3d x-ray surface volume. Each file contains a <b>greyscale slice in the z-direction</b>. Each fragment contains <b>65 slices</b>. Combined this image stack gives us <code>65 * height * width</code> number of voxels per fragment.</p>

In [ ]:
third_train_volume = CFG.train_volumes_path_template.format(3)

In [ ]:
train_volume_3 = load_volume(third_train_volume)

In [ ]:
print(f"3st train volume size: {color.BOLD}{color.CYAN}{train_volume_3[0].shape}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ The whole volume is huge, let's see exactly how much.</p>

In [ ]:
bytes_size = CFG.layers_count * train_volume_3[0].size * train_volume_3[0].itemsize
print(f"Memory size of 3st train volume in bytes: {color.BOLD}{color.PURPLE}{bytes_size}{color.END} B")
print(f"Memory size of 3st train volume in gigabytes: {color.BOLD}{color.PURPLE}{bytes_size / 1024**3}{color.END} GB")

<p style="font-family: consolas; font-size: 16px;">⚪ For a better understanding of how the layers relate to each other, I made an <b>animation with the transition from one layer to another</b>. And so for all 65 layers.</p>

In [ ]:
animate_volume(train_volume_3)

<p style="font-family: consolas; font-size: 16px;">⚪ Let's animate volume right next to its infrared image. So you can look at some features on the fly!</p>

In [ ]:
animate_volume(third_train_volume, ir_photo=train_3_ir)

<p style="font-family: consolas; font-size: 16px;">⚪ And let's animate volume next to its ink labels mask.</p>

In [ ]:
# animate_volume(third_train_volume, inklabels=train_3_inklabels)

<a id="5"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 5. Overview volumes of the <i>test/</i> directory</b></div>

<a id="5.1"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 5.1 Consider <i>mask.png</i> images</b></div>

<p style="font-family: consolas; font-size: 16px;">⚪ Let's load masks</p>

In [ ]:
test_a_mask = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/test/a/mask.png")
test_b_mask = Image.open("/kaggle/input/vesuvius-challenge-ink-detection/test/b/mask.png")

In [ ]:
print(f"Mask a image size: {color.BOLD}{color.CYAN}{test_a_mask.size}{color.END}")
print(f"Mask b image size: {color.BOLD}{color.CYAN}{test_b_mask.size}{color.END}")

<p style="font-family: consolas; font-size: 16px;">⚪ And plot them</p>

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(test_a_mask);

In [ ]:
fig, ax = plt.subplots(dpi=160)
ax.axis('off')
ax.imshow(test_b_mask);

<p style="font-family: consolas; font-size: 16px;">⚪ Let's combine them and you'll see that it's just divided 1st train volume.</p>

In [ ]:
concatenated_test_mask = get_concat_v(test_a_mask, test_b_mask)

In [ ]:
fig, axarr = plt.subplots(2, dpi=160)

axarr[0].imshow(concatenated_test_mask)
axarr[0].axis('off')
axarr[0].set_title("Concatenated test mask")

axarr[1].imshow(train_1_mask)
axarr[1].set_title("1st train volume mask")
axarr[1].axis('off');

In [ ]:
# (⁠ ⁠ꈍ⁠ᴗ⁠ꈍ⁠) WORK STILL IN PROGRESS

# <div style="box-shadow: rgba(240, 46, 170, 0.4) -5px 5px inset, rgba(240, 46, 170, 0.3) -10px 10px inset, rgba(240, 46, 170, 0.2) -15px 15px inset, rgba(240, 46, 170, 0.1) -20px 20px inset, rgba(240, 46, 170, 0.05) -25px 25px inset; padding:20px; font-size:30px; font-family: consolas; display:fill; border-radius:15px; color: rgba(240, 46, 170, 0.7)"> <b> ༼⁠ ⁠つ⁠ ⁠◕⁠‿⁠◕⁠ ⁠༽⁠つ Thank You!</b></div>

<p style="font-family:verdana; color:rgb(34, 34, 34); font-family: consolas; font-size: 16px;"> 💌 Thank you for taking the time to read through my notebook. I hope you found it interesting and informative. If you have any feedback or suggestions for improvement, please don't hesitate to let me know in the comments. <br><br> 🚀 If you liked this notebook, please consider upvoting it so that others can discover it too. Your support means a lot to me, and it helps to motivate me to create more content in the future. <br><br> ❤️ Once again, thank you for your support, and I hope to see you again soon!</p>